## **1. Research Questions introduction**



	“Quais são os fatores (género, número de ratings, e popularidade) que mais influenciam a média de avaliação dos filmes no MovieLens?”

Isto permite:
	•	Explorar a estrutura e a distribuição dos dados (EDA);
	•	Criar métricas derivadas (popularidade, número de votos, média ponderada);
	•	Usar modelos simples (regressão linear / árvore de decisão) para quantificar o impacto de cada variável;
	•	E escalar a análise — localmente em DuckDB, e facilmente replicável em cloud (Athena/BigQuery).

## **2. Pipeline**

### **2.1. Import necessary libraries and packages**

In [ ]:

import sys
print(sys.executable)

#"C:\Users\SaraEstevesHenriques\AppData\Local\Programs\Python\Python313\python.exe" -m pip install polars

c:\Users\SaraEstevesHenriques\AppData\Local\Programs\Python\Python313\python.exe


In [ ]:
import polars as pl
import sys
import os

In [22]:
print ("This is the file directory:", os.getcwd())
parquet_path = os.path.join(os.getcwd(), "moviedetails.parquet")

print("Python executable:", sys.executable)
print("Parquet exists:", os.path.exists(parquet_path))


This is the file directory: c:\Users\SaraEstevesHenriques\Documents\GitHub\BDF25_7\.git\BDF25_7\BDF25_7_data\big_data\ml-32m
Python executable: c:\Users\SaraEstevesHenriques\AppData\Local\Programs\Python\Python313\python.exe
Parquet exists: True


In [23]:
# Eager read (loads into memory)
try:
    df = pl.read_parquet(parquet_path)
    print("Eager read shape:", df.shape)
    display(df.head())
except Exception as e:
    print("Eager read failed:", e)

Eager read shape: (9255, 6)


movieId,title,genres,average_rating,rating_count,tag_count_per_movie
i64,str,str,f64,i64,i64
161582,"""Hell or High Water (2016)""","""Crime|Drama""",3.5625,3176,3176
1982,"""Halloween (1978)""","""Horror""",3.722222,8586,8586
52245,"""Blades of Glory (2007)""","""Comedy|Romance""",3.088235,2482,2482
62,"""Mr. Holland's Opus (1995)""","""Drama""",3.70625,6800,6800
508,"""Philadelphia (1993)""","""Drama""",3.613636,27324,27324


In [ ]:
# Lazy read (better for very large datasets)
try:
    lf = pl.scan_parquet(parquet_path)
    print("LazyFrame created. Example - first 5 rows:")
    print(lf.limit(5).collect())
except Exception as e:
    print("Lazy read failed:", e)

### **2.1. Data Preparation**

2.	Calcular:

•nº total de utilizadores, filmes e avaliações
•distribuição de rating (média, mediana, desvio padrão)
•nº de filmes por género
•nº médio de ratings por filme e por utilizador

3.	Visualizar:
	
•histograma das classificações
•top 10 géneros mais avaliados
•relação entre nº de ratings e média por filme


In [28]:
df_exploded = df.with_columns(
    pl.col("genres").str.split("|")
).explode("genres")

df_exploded.head()

movieId,title,genres,average_rating,rating_count,tag_count_per_movie
i64,str,str,f64,i64,i64
161582,"""Hell or High Water (2016)""","""Crime""",3.5625,3176,3176
161582,"""Hell or High Water (2016)""","""Drama""",3.5625,3176,3176
1982,"""Halloween (1978)""","""Horror""",3.722222,8586,8586
52245,"""Blades of Glory (2007)""","""Comedy""",3.088235,2482,2482
52245,"""Blades of Glory (2007)""","""Romance""",3.088235,2482,2482


### **2.2. Exploratory data analysis**

### **2.3. Feature Engineering**

Criar novas variáveis:
	•	n_ratings → nº de avaliações por filme
	•	avg_rating → média de rating por filme
	•	genre_count → nº de géneros atribuídos
	•	popularity_score = log(1 + nº de ratings) × média (para normalizar popularidade)

## **3. Models**

Objetivo: medir o peso de cada fator sobre a média de avaliação.
Modelos possíveis:
	•	Regressão Linear (OLS) → prever avg_rating com base em n_ratings, genre_count e dummies dos géneros principais.
	•	Árvore de decisão / Random Forest → avaliar a importância relativa das variáveis.
Métricas: R², RMSE, importância de features.


## **4. Visualization and Anaysis**

Gráficos:
	•	Scatter n_ratings vs avg_rating (com tendência)
	•	Boxplots de avg_rating por género
	•	Importância das features no modelo
	•	Insights:
	•	Géneros com maiores médias
	•	Géneros mais populares
	•	Relação entre popularidade e qualidade percebida


## **5. Escalability and big_data**

Executar as queries principais em DuckDB e exportar para Parquet.
	•	Demonstrar (mesmo que parcialmente) a execução de consultas analíticas escaláveis:

SELECT genre, COUNT(*) AS n_filmes, AVG(rating) AS avg_rating
FROM ratings
JOIN movies USING(movieId)
GROUP BY genre;

	•	Mostrar que a mesma lógica pode ser usada em Athena / BigQuery com datasets maiores (ex: MovieLens 25M).

## **5. Conclusions**

	•	Identificar que fatores explicam melhor as boas avaliações.
	•	Propor extensão:
	•	Modelo preditivo por utilizador (colaborativo)
	•	Integração com tags para enriquecer o conteúdo


In [ ]:
# Read a Parquet file with Polars (change the path below)
import sys
import os
import polars as pl

parquet_path = r"C:\path\to\your\file.parquet"  # <-- update this

print("Python executable:", sys.executable)
print("Parquet exists:", os.path.exists(parquet_path))

# Eager read (loads into memory)
try:
    df = pl.read_parquet(parquet_path)
    print("Eager read shape:", df.shape)
    display(df.head())
except Exception as e:
    print("Eager read failed:", e)

# Lazy read (better for very large datasets)
try:
    lf = pl.scan_parquet(parquet_path)
    print("LazyFrame created. Example - first 5 rows:")
    print(lf.limit(5).collect())
except Exception as e:
    print("Lazy read failed:", e)

# If the above fails with ModuleNotFoundError, install polars using the Python shown above:
# "C:\full\path\to\python.exe" -m pip install --upgrade pip
# "C:\full\path\to\python.exe" -m pip install polars
# And register the kernel if needed: python -m pip install ipykernel; python -m ipykernel install --user --name myenv --display-name "Python (myenv)"
